#Data Scrubbing and Pre-processing
##**An example with real mortgage data**


###Outline
This module will focus on the mechanics of the follow data transformations and not their economic/analytic rationale (that will be covered in ____HERE___ ).

In this module we scrub and pre-process raw mortgage data in preparation for regression analysis. This will involve:
1. filling missing values;
2. transforming raw covariates;
3. filtered joins

###Data Sources
-  **Mortgage Origination and Performance data** is taken from the publicly available Fannie Mae Single Family Loan Performance database  http://www.fanniemae.com/portal/funding-the-market/data/loan-performance-data.html. We extract origiation and performance data for the 10k loans of 2005-Q1 acquisitions.
-  **Monthly State HPI data** is taken from Freddie Mac's Home Price Index (HPI) archive http://www.freddiemac.com/finance/fmhpi/archive.html.
-  **Monthly Mortgage Benchmark Rate data** is taken from Freddie Mac's Historical Monthly 30 year Fixed Rate index, http://www.freddiemac.com/pmms/pmms_archives.html.


###Data Description and Roadmap
1. Missing values will be filled with the mean value of similar mortgages that are not missing that value. For example, to impute a missing DTI (debt-to-income) ratio we bucket loans by credit score and loan interest rate and calculate their mean. We similarly bucket other characteristics to impute other missing characteristics. There is a trade-off in bucket width between specificity and idiosyncracy. We want our buckets to contain informative comparables, but not be so granular as to impute idiosyncratic or noisy values.  
2. Covariate transformation includes a) converting a loan's state to a binary value indicating whether the property is in California or Florida, b) calculate the difference between the loan's interest rate and the prevailing Monthly Mortgage Benchmark Rate  
3. For each mortgage we would like to:  
  1. calculate the lowest HPI expressed as a % of starting value, for for each of three 2-year intervals from the loan's origination date, i.e the lowest HPI reached, divided by starting HPI over a) Years 1 & 2, b) Years 3 & 4, c) Years 5 & 6.  
  2. We would also like to similarly calculate the greatest absolute reduction in the Monthly Mortgage Benchmark Rate over each of three 2-year intervals from the loan's origination.
  3. We also want to create a response variable identifying whether the loan exited during intervals (0,2], (2,4], (4,6] and if so, when and for what reason, i.e. prepay or default.

###Let's go...

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

We load our data sources, preview them, and select the columns and create our base table to which we'll add columns.

In [15]:
Orig = pd.read_csv("2005-1-Orig.csv")
Perf = pd.read_csv("2005-1-Perf.csv")
Rates = pd.read_csv("BenchmarkRatesto2016.csv")
HPI = pd.read_csv("HPI.csv")

Orig.head()
Perf.head()
Rates.head()
HPI.head()

Log_Orig = Orig[["LOAN_ID","ORIG_AMT","ORIG_RT","ORIG_DTE","OCLTV","DTI","CSCORE_B","STATE"]].copy()

##1. Missing Values

In [16]:
Log_Orig[Log_Orig['OCLTV'].isnull() | Log_Orig['ORIG_RT'].isnull() | Log_Orig['CSCORE_B'].isnull() | Log_Orig['DTI'].isnull()].describe()

,LOAN_ID,ORIG_AMT,ORIG_RT,OCLTV,DTI,CSCORE_B
count,8.010000e+02,801.000000,801.000000,799.000000,84.000000,709.000000
mean,5.485218e+11,175210.986267,5.770473,72.279099,34.952381,715.623413
std,2.561726e+11,83522.447157,0.308034,16.674037,12.063991,60.735888
min,1.001936e+11,25000.000000,5.000000,7.000000,4.000000,473.000000
25%,3.296775e+11,113000.000000,5.500000,65.000000,26.750000,674.000000
50%,5.523753e+11,159000.000000,5.750000,76.000000,36.000000,723.000000
75%,7.663707e+11,232000.000000,5.875000,80.000000,43.500000,765.000000
max,9.994329e+11,556000.000000,6.750000,104.000000,64.000000,821.000000


By looking at the first row, '**count**', we see there are 801 loans with one or more missing value. All 801 loans with missing values do have values for ORIG_RT, 2 loans (801 minus 799) are missing OCLTV, 717 loans (801 minus 84) are missing DTI, and 92 (801 minus 709) are missing CSCORE_B.

###Populating Missing Data

We'll show a method of bucketing the loans on-the-fly (we could instead have created additional columns indicating the bucket of each loan and run a groupby on those columns), so we pass the groupby method a function describing how to bucket each loan.

Based on these groupings we run the fillna method to set missing values to the mean of the respective group.

This method should catch all missing values, except where missing values are alone in their designated bucket without any other loans from which to impute a value.

In [17]:
# Create helper function used to bucket each loan
def Buckets(table,x, fillVar):

    if (fillVar == 'CSCORE'): # for a missing CSCORE, bucket by OCLTV and ORIG_RT
        (OCLTV,Rate) = (round(table['OCLTV'].loc[x]/20,0)*20, round(table['ORIG_RT'].loc[x],0))
        if np.isnan(OCLTV): OCLTV = round(table['OCLTV'].mean()/20,0)*20
        return "OCLTV-{}/Rate-{}".format(OCLTV,Rate)
    
    elif (fillVar == 'DTI') or (fillVar == 'OCLTV'): #for a missing DTI or OCLTV, bucket by CSCORE and ORIG_RT
        (FICO,Rate) = (round(table['CSCORE_B'].loc[x]/50,0)*50, round(table['ORIG_RT'].loc[x],0))
        if np.isnan(FICO): FICO = round(table['CSCORE_B'].mean()/50,0)*50
        return "FICO-{}/Rate-{}".format(FICO,Rate)


# Pass the helper function to the groupby method, and tell it to fill NAs using the mean of that bucket
fill_mean = lambda x: x.fillna(x.mean())

Log_Orig['CSCORE_B'] = Log_Orig.groupby(lambda x: Buckets(Log_Orig, x, 'CSCORE'))['CSCORE_B'].apply(fill_mean)
Log_Orig['DTI'] = Log_Orig.groupby(lambda x: Buckets(Log_Orig, x, 'DTI'))['DTI'].apply(fill_mean)
Log_Orig['OCLTV'] = Log_Orig.groupby(lambda x: Buckets(Log_Orig, x, 'OCLTV'))['OCLTV'].apply(fill_mean)

Let's check to see what missing values remain

In [18]:
Log_Orig[Log_Orig['OCLTV'].isnull() | Log_Orig['ORIG_RT'].isnull() | Log_Orig['CSCORE_B'].isnull() | Log_Orig['DTI'].isnull()]

,LOAN_ID,ORIG_AMT,ORIG_RT,ORIG_DTE,OCLTV,DTI,CSCORE_B,STATE
8090,830690271521,55000,6.25,01/2005,39,NaN,473,FL


We could rerun our stratification approach using slightly coarser buckets, but for simplicity we will impute the sample mean

In [19]:
Log_Orig['DTI'] = Log_Orig['DTI'].fillna(Log_Orig['DTI'].mean())

##2. Transforming Covariates

Since we will be performing date arthimetic we must convert the relevant dates into the datetime format.

Identifying loans originated in California or Florida is a simple call.

Calculating the Spread-AT-Origination (SATO) for each loan requires joining each loan with the Monthly Mortgage Benchmark Rate table on the origination date, and calculating the difference between the loan rate and the prevailing Benchmark rate

In [20]:
Log_Orig['ORIG_DTE'] = pd.to_datetime(Log_Orig['ORIG_DTE'], format='%m/%Y') #format to datetime

Rates.set_index(pd.to_datetime(Rates['Date']), inplace=True)
HPI.set_index(pd.to_datetime(HPI['Month']), inplace=True)

#CA-FL identifier
Log_Orig["CA-FL"] = np.where(Log_Orig["STATE"].isin(["CA","FL"]),1,0)

#SATO Calculation
Log_Orig = Log_Orig.merge(Rates, how='left', left_on='ORIG_DTE', right_index=True)
Log_Orig.rename(columns={'Rate':'ORIG_BENCHMK'}, inplace=True)

Log_Orig['SATO'] = Log_Orig['ORIG_RT'] - Log_Orig['ORIG_BENCHMK']

##3. Filtered Joins

There are 2 states for which we do not have HPI data; Puerto Rico and the Virgin Islands. Since these comprise a small portion of our pool we will simply reclassify them as Hawaii.

In [21]:
Log_Orig.ix[Log_Orig['STATE'] == 'PR','STATE'] = 'HI'
Log_Orig.ix[Log_Orig['STATE'] == 'VI','STATE'] = 'HI'

We create a function that takes a loan's origination date and State, and computes the lowest HPI as a percent of starting HPI, given an interval duration and offset. The function also computes similar values for the Benchmark rate.

We map the function to our origination series and have it populate our original table.

In [22]:
import datetime as dt
def minVal(series,origdate,offset,duration,state):

    startdate = dt.date(origdate.year+offset, origdate.month, origdate.day)
    enddate = dt.date(origdate.year+offset+duration, origdate.month, origdate.day)
    
    if series=='HPI':
        return min(HPI.ix[startdate:enddate,state]) / HPI.ix[startdate,state]
    if series=='Rate':
        return Rates.ix[startdate,'Rate'] - min(Rates.ix[startdate:enddate,'Rate'])

In [23]:
#Populate intervals with lowest HPI value
Log_Orig['HPI-Y1&2']=map(lambda x, y: minVal('HPI',x,0,2,y), Log_Orig['ORIG_DTE'], Log_Orig['STATE'])
Log_Orig['HPI-Y3&4']=map(lambda x, y: minVal('HPI',x,2,2,y), Log_Orig['ORIG_DTE'], Log_Orig['STATE'])
Log_Orig['HPI-Y5&6']=map(lambda x, y: minVal('HPI',x,4,2,y), Log_Orig['ORIG_DTE'], Log_Orig['STATE'])

#Populate intervals with lowest benchmark rate value
Log_Orig['Rate-Y1&2']=map(lambda x, y: minVal('Rate',x,0,2,y), Log_Orig['ORIG_DTE'], Log_Orig['STATE'])
Log_Orig['Rate-Y3&4']=map(lambda x, y: minVal('Rate',x,2,2,y), Log_Orig['ORIG_DTE'], Log_Orig['STATE'])
Log_Orig['Rate-Y5&6']=map(lambda x, y: minVal('Rate',x,4,2,y), Log_Orig['ORIG_DTE'], Log_Orig['STATE'])

###Adding a Response Variable

We have spent alot of effort preparing the covariate data, filling missing values, and attaching macroeconomic data. Let's now turn our attention to the response variable, which to fill we need to look at the performance data table.

We write a function that returns the date and reason for a loan's exit. The __Zero Balance Code__ is Fannie Mae's way of encoding an event that causes a loan to be removed from the pool - prepayment/refinance, or a variety of different liquidation pathways like short sale or deed-in-lieu. Natural payment is also an option but since these are 30 year mortgages we do not have enough data.

In addition to extracting the default date and type, we encode it into variable that tells us which if any of the first three 2-year intervals the event falls into.

In [24]:
def extractDefault(ID):
    a = Perf.ix[(Perf['LOAN_ID']==ID) & Perf['ZB_DTE'].notnull(),['Zero.Bal.Code','ZB_DTE']]
    if len(a) == 0: return np.NAN,np.NAN
    else: return a.iloc[0,0], a.iloc[0,1]
    
Log_Orig['DEFAULT.CODE'], Log_Orig['DEFAULT.DATE'] = zip(*map(extractDefault,Log_Orig['LOAN_ID']))

Log_Orig['DEFAULT.DATE'] = pd.to_datetime(Log_Orig['DEFAULT.DATE'])

In [25]:
def ResponseValue(Code,OrigDate,DefDate):
    if Code in [3,6,9]:
        DefDate = dt.date(DefDate.year, DefDate.month, DefDate.day)
        
        if DefDate <= dt.date(OrigDate.year+2, OrigDate.month, OrigDate.day):
            return 1
        elif DefDate <= dt.date(OrigDate.year+4, OrigDate.month, OrigDate.day):
            return 2
        elif DefDate <= dt.date(OrigDate.year+6, OrigDate.month, OrigDate.day):
            return 3
    return 0
        
Log_Orig['RESPONSE'] = map(ResponseValue, Log_Orig['DEFAULT.CODE'], Log_Orig['ORIG_DTE'], Log_Orig['DEFAULT.DATE'])

##Conclusion
Well that's it. We've taken raw mortgage data, cleaned it, transformed some of it, and appended relevant macroeconomic changes preparing it for use in regression analysis. Please see Part 2 as the adventure continues. There we will explore why we selected and transformed the variables we did, and interpret a logistic regression run on the data.